**Global imports**

In [3]:
import numpy as np

In [153]:
import scipy.stats as st

In [4]:
import shelve

In [274]:
import math

In [327]:
from itertools import ifilter

**Database stuff**

In [5]:
db = shelve.open("./db/movielens_100k.db")

In [6]:
# db.close()

**Intersecting critic rated movies**

In [275]:
def shared_items_original(db, person1, person2):
    return [item for item in db[person1] if item in db[person2]]

In [276]:
%time shared_items_original(db, "1", "2")

CPU times: user 9.75 ms, sys: 4.14 ms, total: 13.9 ms
Wall time: 13.5 ms


['Birdcage, The (1996)',
 'Toy Story (1995)',
 'Postino, Il (1994)',
 'Good Will Hunting (1997)',
 'Richard III (1995)',
 'Star Wars (1977)',
 'Jerry Maguire (1996)',
 'Godfather, The (1972)',
 'Full Monty, The (1997)',
 'Shall We Dance? (1996)',
 'Kolya (1996)',
 'Contact (1997)',
 'Truth About Cats & Dogs, The (1996)',
 'Men in Black (1997)',
 'Mighty Aphrodite (1995)',
 "Antonia's Line (1995)",
 'Fargo (1996)',
 "My Best Friend's Wedding (1997)"]

In [277]:
def shared_items_optimized(db, person1, person2):
    return db[person1].viewkeys() & db[person2].viewkeys()

In [278]:
%time shared_items_optimized(db, "1", "2")

CPU times: user 1.53 ms, sys: 125 µs, total: 1.65 ms
Wall time: 1.22 ms


{"Antonia's Line (1995)",
 'Birdcage, The (1996)',
 'Contact (1997)',
 'Fargo (1996)',
 'Full Monty, The (1997)',
 'Godfather, The (1972)',
 'Good Will Hunting (1997)',
 'Jerry Maguire (1996)',
 'Kolya (1996)',
 'Men in Black (1997)',
 'Mighty Aphrodite (1995)',
 "My Best Friend's Wedding (1997)",
 'Postino, Il (1994)',
 'Richard III (1995)',
 'Shall We Dance? (1996)',
 'Star Wars (1977)',
 'Toy Story (1995)',
 'Truth About Cats & Dogs, The (1996)'}

In [279]:
shared_items = shared_items_optimized

**Vectorizing ratings**

In [280]:
def critics_ratings_original(db, person1, person2):
    shared = shared_items(db, person1, person2)
    scores1 = [db[person1][item] for item in shared]
    scores2 = [db[person2][item] for item in shared]
    n = len(shared)
    return scores1, scores2, n

In [281]:
%timeit s1, s2, n = critics_ratings_original(db, "1", "2")

The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 3.29 ms per loop


In [282]:
def critics_ratings_optimized(db, person1, person2):
    shared = shared_items(db, person1, person2)
    n = len(shared)
    scores1 = np.fromiter(map(lambda x: db[person1][x], shared), np.float_)
    scores2 = np.fromiter(map(lambda x: db[person2][x], shared), np.float_)
    return scores1, scores2, n

In [283]:
%timeit s1, s2, n = critics_ratings_optimized(db, "1", "2")

100 loops, best of 3: 3.26 ms per loop


In [284]:
critics_ratings = critics_ratings_optimized

**Similarity metrics - Euclidean**

In [285]:
def euclidean_similarity_original(db, person1, person2):
    s1, s2, n = critics_ratings(db, person1, person2)
    dist = math.sqrt(sum(map(lambda x, y: pow(x-y, 2), s1, s2)))
    return 1/(1+dist)

In [286]:
%time euclidean_similarity_original(db, "1", "2")

CPU times: user 7.37 ms, sys: 0 ns, total: 7.37 ms
Wall time: 7.29 ms


0.15894454156034005

In [287]:
def euclidean_similarity_optimized(db, person1, person2):
    s1, s2, n = critics_ratings(db, person1, person2)
    return 1/(1+np.linalg.norm(s1-s2))

In [288]:
%time euclidean_similarity_optimized(db, "1", "2")

CPU times: user 4.18 ms, sys: 15 µs, total: 4.2 ms
Wall time: 3.96 ms


0.15894454156034005

In [289]:
euclidean_similarity = euclidean_similarity_optimized

**Similarity metrics - Pearson**

In [290]:
def pearson_similarity_original(db, person1, person2):
    s1, s2, n = critics_ratings(db, person1, person2)
    if n == 0: return 0
    sum_s1 = sum(s1)
    sum_s2 = sum(s2)
    sum_s1_sq = sum(map(lambda x,y:x*y, s1, s1))
    sum_s2_sq = sum(map(lambda x,y:x*y, s2, s2))
    sum_s_pro = sum(map(lambda x,y:x*y, s1, s2))
    numerator = sum_s_pro - (sum_s1*sum_s2/float(n))
    denominator = sqrt((sum_s1_sq-pow(sum_s1, 2)/float(n))*
                       (sum_s2_sq-pow(sum_s2, 2)/float(n)))
    if denominator == 0: return 0
    return numerator/denominator

In [291]:
%time pearson_similarity_original(db, "900", "800")

CPU times: user 1.55 ms, sys: 125 µs, total: 1.68 ms
Wall time: 1.27 ms


-0.55901699437494845

In [292]:
def pearson_similarity_optimized(db, person1, person2):
    s1, s2, n = critics_ratings(db, person1, person2)
    if n == 0: return 0
    return st.pearsonr(s1, s2)[0]

In [293]:
%time pearson_similarity_optimized(db, "900", "800")

CPU times: user 912 µs, sys: 75 µs, total: 987 µs
Wall time: 749 µs


-0.55901699437494745

In [294]:
def pearson_similarity_optimized2(db, person1, person2):
    s1, s2, n = critics_ratings(db, person1, person2)
    sum_s1 = np.sum(s1)
    sum_s2 = np.sum(s2)
    sum_s1_sq = np.sum(s1*s1)
    sum_s2_sq = np.sum(s2*s2)
    sum_s_pro = np.sum(s1*s2)
    numerator = sum_s_pro - (sum_s1*sum_s2/float(n))
    denominator = np.sqrt((sum_s1_sq-(sum_s1*sum_s1)/float(n))*
                          (sum_s2_sq-(sum_s2*sum_s2)/float(n)))
    if denominator == 0: return 0
    return numerator/denominator

In [295]:
%time pearson_similarity_optimized2(db, "900", "800")

CPU times: user 916 µs, sys: 75 µs, total: 991 µs
Wall time: 1.05 ms


-0.55901699437494845

In [296]:
pearson_similarity = pearson_similarity_optimized

**Recommend critics based on all the db**

In [304]:
def recommend_critics_original(db, person, similarity):
    lst = [(similarity(db, person, other), other)
           for other in db if other != person]
    cleaned = filter(lambda p: p[0]==p[0], lst)
    return sorted(cleaned)[::-1]

In [332]:
%time recommend_critics_original(db, "1", pearson_similarity)

CPU times: user 9.23 s, sys: 690 ms, total: 9.92 s
Wall time: 9.94 s


[(1.0, '866'),
 (1.0, '812'),
 (1.0, '811'),
 (1.0, '810'),
 (1.0, '531'),
 (1.0, '511'),
 (1.0, '39'),
 (1.0, '351'),
 (1.0, '273'),
 (1.0, '166'),
 (0.95257934441568026, '520'),
 (0.91653421378307787, '107'),
 (0.90453403373329089, '687'),
 (0.89104211121363053, '34'),
 (0.88229880338305944, '105'),
 (0.8703882797784892, '740'),
 (0.86602540378443871, '485'),
 (0.86602540378443849, '873'),
 (0.86602540378443849, '400'),
 (0.86258194917794284, '510'),
 (0.85280286542244166, '364'),
 (0.8496546792899824, '803'),
 (0.84548890303097124, '791'),
 (0.84366148773210736, '572'),
 (0.83874169727605896, '691'),
 (0.81649658092772615, '111'),
 (0.81409157841069435, '61'),
 (0.81064348337777759, '754'),
 (0.79494818839587555, '702'),
 (0.77777777777777779, '656'),
 (0.77777777777777779, '240'),
 (0.7745966692414834, '628'),
 (0.77174363314128969, '134'),
 (0.7660323462854266, '736'),
 (0.76187297467052739, '433'),
 (0.76058270804488615, '672'),
 (0.75234638921740205, '550'),
 (0.75, '920'),
 (0.

In [328]:
def recommend_critics_optimized(db, person, similarity):
    lst = [(similarity(db, person, other), other)
           for other in db if other != person]
    cleaned = ifilter(lambda p: p[0]==p[0], lst)
    return sorted(cleaned)[::-1]

In [333]:
%time recommend_critics_optimized(db, "1", pearson_similarity)

CPU times: user 9.27 s, sys: 603 ms, total: 9.87 s
Wall time: 9.89 s


[(1.0, '866'),
 (1.0, '812'),
 (1.0, '811'),
 (1.0, '810'),
 (1.0, '531'),
 (1.0, '511'),
 (1.0, '39'),
 (1.0, '351'),
 (1.0, '273'),
 (1.0, '166'),
 (0.95257934441568026, '520'),
 (0.91653421378307787, '107'),
 (0.90453403373329089, '687'),
 (0.89104211121363053, '34'),
 (0.88229880338305944, '105'),
 (0.8703882797784892, '740'),
 (0.86602540378443871, '485'),
 (0.86602540378443849, '873'),
 (0.86602540378443849, '400'),
 (0.86258194917794284, '510'),
 (0.85280286542244166, '364'),
 (0.8496546792899824, '803'),
 (0.84548890303097124, '791'),
 (0.84366148773210736, '572'),
 (0.83874169727605896, '691'),
 (0.81649658092772615, '111'),
 (0.81409157841069435, '61'),
 (0.81064348337777759, '754'),
 (0.79494818839587555, '702'),
 (0.77777777777777779, '656'),
 (0.77777777777777779, '240'),
 (0.7745966692414834, '628'),
 (0.77174363314128969, '134'),
 (0.7660323462854266, '736'),
 (0.76187297467052739, '433'),
 (0.76058270804488615, '672'),
 (0.75234638921740205, '550'),
 (0.75, '920'),
 (0.

In [334]:
recommend_critics = recommend_critics_optimized

In [335]:
def top_critics(db, person, similarity, n=30):
    return recommend_critics(db, person, similarity)[:n]

In [336]:
%time top_critics(db, "1", pearson_similarity)

CPU times: user 9.87 s, sys: 595 ms, total: 10.5 s
Wall time: 10.5 s


[(1.0, '866'),
 (1.0, '812'),
 (1.0, '811'),
 (1.0, '810'),
 (1.0, '531'),
 (1.0, '511'),
 (1.0, '39'),
 (1.0, '351'),
 (1.0, '273'),
 (1.0, '166'),
 (0.95257934441568026, '520'),
 (0.91653421378307787, '107'),
 (0.90453403373329089, '687'),
 (0.89104211121363053, '34'),
 (0.88229880338305944, '105'),
 (0.8703882797784892, '740'),
 (0.86602540378443871, '485'),
 (0.86602540378443849, '873'),
 (0.86602540378443849, '400'),
 (0.86258194917794284, '510'),
 (0.85280286542244166, '364'),
 (0.8496546792899824, '803'),
 (0.84548890303097124, '791'),
 (0.84366148773210736, '572'),
 (0.83874169727605896, '691'),
 (0.81649658092772615, '111'),
 (0.81409157841069435, '61'),
 (0.81064348337777759, '754'),
 (0.79494818839587555, '702'),
 (0.77777777777777779, '656')]

**Recommend movies based on all the db**

In [341]:
def recommend_movies_original(db, person, similarity, n=30, m=30):
    totals = {}
    similarity_sums = {}
    for simil, critic in top_critics(db, person, similarity, n):
        if simil <= 0: continue
        for movie in db[critic]:
            if movie not in db[person] or db[person][movie] == 0:
                totals.setdefault(movie, 0)
                totals[movie] += db[critic][movie]*simil
                similarity_sums.setdefault(movie, 0)
                similarity_sums[movie] += simil
    rankings = [(total/similarity_sums[movie], movie)
               for movie, total in totals.items()]
    return sorted(rankings)[::-1][:m]

In [353]:
%time recommend_movies_original(db, "1", pearson_similarity)

CPU times: user 9.49 s, sys: 623 ms, total: 10.1 s
Wall time: 10.1 s


[(5.0000000000000009, 'Postman, The (1997)'),
 (5.0, 'Women, The (1939)'),
 (5.0, 'Winter Guest, The (1997)'),
 (5.0, "She's the One (1996)"),
 (5.0, 'Shadow Conspiracy (1997)'),
 (5.0, 'Seventh Seal, The (Sjunde inseglet, Det) (1957)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, "Schindler's List (1993)"),
 (5.0, 'Rear Window (1954)'),
 (5.0, 'Philadelphia (1993)'),
 (5.0, 'Matilda (1996)'),
 (5.0, 'Love Jones (1997)'),
 (5.0, 'Leaving Las Vegas (1995)'),
 (5.0, 'Kids (1995)'),
 (5.0, "It's a Wonderful Life (1946)"),
 (5.0, 'It Happened One Night (1934)'),
 (5.0, 'Great Dictator, The (1940)'),
 (5.0, 'Big Lebowski, The (1998)'),
 (5.0, 'American President, The (1995)'),
 (4.5522898401481484, 'Thousand Acres, A (1997)'),
 (4.3717778769846021, 'Desperate Measures (1998)'),
 (4.2753227231893352, 'Titanic (1997)'),
 (4.2733948756213609, 'Rainmaker, The (1997)'),
 (4.2148784307905158, 'Anna Karenina (1997)'),
 (4.0598326384530221, 'L.A. Confidential (1997)'),
 (4.0275325817142607, 'Mrs

In [357]:
def recommend_movies_optimized(db, person, similarity, n=30, m=30):
    totals = {}
    similarity_sums = {}
    for simil, critic in iter(top_critics(db, person, similarity, n)):
        if simil <= 0: continue
        for movie in db[critic].iterkeys():
            if movie not in db[person]:
                totals.setdefault(movie, 0)
                totals[movie] += db[critic][movie]*simil
                similarity_sums.setdefault(movie, 0)
                similarity_sums[movie] += simil
    rankings = [(total/similarity_sums[movie], movie)
               for movie, total in totals.items()]
    return sorted(rankings)[::-1][:m]

In [358]:
%time recommend_movies_optimized(db, "1", pearson_similarity)

CPU times: user 9.33 s, sys: 563 ms, total: 9.89 s
Wall time: 9.91 s


[(5.0000000000000009, 'Postman, The (1997)'),
 (5.0, 'Women, The (1939)'),
 (5.0, 'Winter Guest, The (1997)'),
 (5.0, "She's the One (1996)"),
 (5.0, 'Shadow Conspiracy (1997)'),
 (5.0, 'Seventh Seal, The (Sjunde inseglet, Det) (1957)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, "Schindler's List (1993)"),
 (5.0, 'Rear Window (1954)'),
 (5.0, 'Philadelphia (1993)'),
 (5.0, 'Matilda (1996)'),
 (5.0, 'Love Jones (1997)'),
 (5.0, 'Leaving Las Vegas (1995)'),
 (5.0, 'Kids (1995)'),
 (5.0, "It's a Wonderful Life (1946)"),
 (5.0, 'It Happened One Night (1934)'),
 (5.0, 'Great Dictator, The (1940)'),
 (5.0, 'Big Lebowski, The (1998)'),
 (5.0, 'American President, The (1995)'),
 (4.5522898401481484, 'Thousand Acres, A (1997)'),
 (4.3717778769846021, 'Desperate Measures (1998)'),
 (4.2753227231893352, 'Titanic (1997)'),
 (4.2733948756213609, 'Rainmaker, The (1997)'),
 (4.2148784307905158, 'Anna Karenina (1997)'),
 (4.0598326384530221, 'L.A. Confidential (1997)'),
 (4.0275325817142607, 'Mrs

In [364]:
get_recommendations = recommend_movies_optimized

**Comparison with current implementation**

In [360]:
import recommendations as rec

In [366]:
%time rec.get_recommendations(db, "1")

CPU times: user 1min 5s, sys: 5.06 s, total: 1min 10s
Wall time: 1min 10s


[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Little City (1998)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.999999999999999, 'Great Day in Harlem, A (1994)'),
 (4.768149552524299, 'Two or Three Things I Know About Her (1966)'),
 (4.682633172633338, 'Anna (1996)'),
 (4.623726612933918, 'Pather Panchali (1955)'),
 (4.521589387248104, 'Close Shave, A (1995)'),
 (4.515144270419964, "Some Mother's Son (1996)"),
 (4.514962379404503, 'Aparajito (1956)'),
 (4.502557827437198, 'Casablanca (1942)'),
 (4.501965635892865, 'Hearts and Minds (1996)'),
 (4.467929758957103, 'Everest (1998)'),
 (4.467775885199613, "Schindler's List (1993)"),
 (4.452311232743258, 'Rear Window (1954)'),
 (4.430827645329046, 'Leading Man, The (1996)'),
 (4.398314267646154, 'Third Man, The 

In [367]:
%time get_recommendations(db, "1", pearson_similarity)

CPU times: user 9.79 s, sys: 599 ms, total: 10.4 s
Wall time: 10.4 s


[(5.0000000000000009, 'Postman, The (1997)'),
 (5.0, 'Women, The (1939)'),
 (5.0, 'Winter Guest, The (1997)'),
 (5.0, "She's the One (1996)"),
 (5.0, 'Shadow Conspiracy (1997)'),
 (5.0, 'Seventh Seal, The (Sjunde inseglet, Det) (1957)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, "Schindler's List (1993)"),
 (5.0, 'Rear Window (1954)'),
 (5.0, 'Philadelphia (1993)'),
 (5.0, 'Matilda (1996)'),
 (5.0, 'Love Jones (1997)'),
 (5.0, 'Leaving Las Vegas (1995)'),
 (5.0, 'Kids (1995)'),
 (5.0, "It's a Wonderful Life (1946)"),
 (5.0, 'It Happened One Night (1934)'),
 (5.0, 'Great Dictator, The (1940)'),
 (5.0, 'Big Lebowski, The (1998)'),
 (5.0, 'American President, The (1995)'),
 (4.5522898401481484, 'Thousand Acres, A (1997)'),
 (4.3717778769846021, 'Desperate Measures (1998)'),
 (4.2753227231893352, 'Titanic (1997)'),
 (4.2733948756213609, 'Rainmaker, The (1997)'),
 (4.2148784307905158, 'Anna Karenina (1997)'),
 (4.0598326384530221, 'L.A. Confidential (1997)'),
 (4.0275325817142607, 'Mrs

In [ ]:
%time get_recommendations(db, "1", euclidean_similarity)